In [29]:
import requests
from bs4 import BeautifulSoup
import json

In [30]:
import pandas as pd
import numpy as np

In [39]:
url = 'https://www.yelp.com/search?'
business = input('What business are you looking for (Restaurants, Hairdressers,...) ? ')
address = input('What is your search area : (city,street,neighborhood,...) ? ')
address = address.replace(' ','+')

url = 'https://www.yelp.com/search?find_desc=%s&find_loc=%s,+NY&start=30' % (business,address)
print(url)

What business are you looking for (Restaurants, Hairdressers,...) ? Restaurants
What is your search area : (city,street,neighborhood,...) ? New York
https://www.yelp.com/search?find_desc=Restaurants&find_loc=New+York,+NY&start=30


In [40]:
response = requests.get(url)
response.status_code

503

In [41]:
results_page = BeautifulSoup(response.content,'lxml')
results_page

<!DOCTYPE html>
<!--[if lt IE 7 ]> <html class="ie6 ie ltie9 ltie8 no-js" lang="en_US"> <![endif]--><!--[if IE 7 ]>    <html class="ie7 ie ltie9 ltie8 no-js" lang="en_US"> <![endif]--><!--[if IE 8 ]>    <html class="ie8 ie ltie9 no-js" lang="en_US"> <![endif]--><!--[if IE 9 ]>    <html class="ie9 ie no-js" lang="en_US"> <![endif]--><!--[if (gt IE 9)|!(IE)]><!--><html class="no-js" lang="en_US"> <!--<![endif]-->
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="https://s3-media1.fl.yelpcdn.com/assets/srv0/error_pages/8a98d2743e15/assets/scss/maintenance-pkg.css" rel="stylesheet"/>
<title>Yelp</title>
<link content="#c41200" href="https://s3-media1.fl.yelpcdn.com/assets/srv0/yelp_styleguide/4374c8fd03d1/assets/img/logos/yelp_burst.svg" rel="mask-icon" sizes="any"/>
<link href="https://s3-media2.fl.yelpcdn.com/assets/srv0/yelp_styleguide/118ff475a341/assets/img/logos/favicon.ico" rel="shortcut icon"/>
</head>
<body>
<div class="main-header main-conten

In [34]:
results_page.find_all('li',class_ = "regular-search-result")

[]

In [35]:
def get_businesses(limit = 30):
    
    loop_range = limit//30 
    
    business_numbers = list((np.arange(loop_range))*30)
    
    for page_index in business_numbers:
    
        df = pd.DataFrame(columns = ['name','open or closed','latitude','longitude','price range','rating value','review_count','categories','phone'])

        business = input('What business are you looking for (Restaurants, Hairdressers,...) ? ')
        address = input('What is your search area : (city,street,neighborhood,...) ? ')
        address = address.replace(' ','+')

        url = 'https://www.yelp.com/search?find_desc=%s&find_loc=%s,+NY&start=%s' % (business,address,page_index)
        print(url)
        response = requests.get(url)
        results_page = BeautifulSoup(response.content,'lxml')
        businesses = results_page.find_all('li',class_ = "regular-search-result")
        i = 0
        l = []
        for business in businesses:
            a_tag = business.find('a',{'class': "biz-name js-analytics-click"})
            business_name = a_tag.find('span').get_text()
            business_link = 'https://www.yelp.com' + a_tag.get('href')
            info_list = get_info(business_link)
            latitude,longitude,category_list,price_range,rating_value,review_count,phone_number,open_ = info_list[0],info_list[1],info_list[2],info_list[3],info_list[4],info_list[5],info_list[6],info_list[7]
            df.loc[i] = [business_name,open_,latitude,longitude,price_range,rating_value,review_count,category_list,phone_number]

            i += 1

        df.set_index('name', inplace = True)

    return(df)

In [156]:
def get_info(link):
    response_business = requests.get(link)
    results_business_page = BeautifulSoup(response_business.content,'lxml')
    
    latitude = 0
    longitude = 0
    category_list = []
    price_range = ''
    rating_value = 0
    review_count = 0
    phone_number = ''
    open_ = ''
        
    json_map = results_business_page.find('div', class_ = "lightbox-map hidden").get('data-map-state')
    dict_map = json.loads(json_map)
    latitude = dict_map["markers"][1]["location"]["latitude"]
    longitude = dict_map["markers"][1]["location"]["longitude"]

    categories = results_business_page.find("span", class_ = 'category-str-list')
    for category in categories.find_all('a'):
        category_list.append(category.get_text())
        
    try:
        price_range = results_business_page.find("span", class_ = 'business-attribute price-range').get_text()
    except:
        price_range = None
        
    ratings = results_business_page.find("div", itemprop = 'aggregateRating')
    rating_value = ratings.find("meta").get('content')
    review_count = ratings.find("span").get_text()
    
    try:
        phone_number = results_business_page.find('span',itemprop = "telephone").get_text().strip()
    except:
        phone_number = None
        
    try:
        open_ = results_business_page.find('span', class_ = "nowrap extra open").get_text()
    except:
        open_ = None
        
    return(latitude,longitude,category_list,price_range,rating_value,review_count,phone_number,open_)

In [157]:
url = 'https://www.yelp.com/biz/bunker-brooklyn?osq=Restaurants'
response_business = requests.get(url)
results_business_page = BeautifulSoup(response_business.content,'lxml')
phone_number = results_business_page.find('span',itemprop = "telephone").get_text()
phone_number.strip()

'(718) 386-4282'

In [24]:
get_info('https://www.yelp.com/biz/bunker-brooklyn?osq=Restaurants')

(40.7105638,
 -73.9233984,
 ['Vietnamese'],
 '$$',
 '4.0',
 '642',
 '(718) 386-4282',
 'Open now')

In [2]:
get_businesses()

NameError: name 'np' is not defined

In [185]:
list((1+np.arange(2))*30)

[30, 60]